In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pyautogui

driver = webdriver.Chrome('C:/WebDriver/chromedriver')

example_ticker = 'safemoon'
ticker = example_ticker

driver.get("https://www.youtube.com/results?search_query=" + ticker)

# while "no more results" is not displayed
while(True):
    try:
        driver.find_element_by_xpath("/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[30]/div[3]/ytd-message-renderer/yt-formatted-string[1]")
        
    except:
        pyautogui.press('pagedown')
        continue
        
    break
        
print("FINISHHHED")

FINISHHHED
